In [53]:
# dependencies and setup
import pandas as pd
from sqlalchemy import create_engine

In [54]:
# Store CSV to DataFrame
listings_file = "resources/listings.csv"
listing_host_df = pd.read_csv(listings_file)

In [75]:
# change column "name" to something that SQL doesn't already recognize
listing_host_df.rename(index=str,columns={"name":"listing_name"},inplace=True)
# listing_host_df.head()
print(listings_host_df.columns)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

## Create "Listings" dataframe and clean it

In [76]:

# Create new data with select columns for the AirBnB listings
listing_df = listing_host_df[["id","listing_name","street","neighbourhood_cleansed",
                              "zipcode","latitude","longitude","property_type","room_type",
                              "accommodates","bathrooms","bedrooms","beds","bed_type","price",
                              "weekly_price","monthly_price","security_deposit","cleaning_fee",
                              "guests_included","extra_people","has_availability","availability_30",
                              "availability_60","availability_90","availability_365","number_of_reviews",
                              "review_scores_rating","review_scores_accuracy","review_scores_cleanliness",
                              "review_scores_checkin","review_scores_communication","review_scores_location",
                              "review_scores_value","requires_license","instant_bookable",
                              "cancellation_policy","host_id"]].copy()

In [77]:
# convert boolean columns to boolean
# listing_df["has_availability"].replace(["t","f"],[True,False],inplace=True)
listing_df["requires_license"].replace(["t","f"],[True,False],inplace=True)
listing_df["instant_bookable"].replace(["t","f"],[True,False],inplace=True)

In [78]:
# convert all the currency columns to numeric values instead of strings
# define the currency columns
currency_cols=["price","weekly_price","monthly_price","security_deposit","cleaning_fee","extra_people"]
# remove dollar sign and commas
listing_df[currency_cols]=listing_df[currency_cols].replace({'\$': '', ',': ''}, regex=True)
# changing the leftover value (minus currency sign and commas) to a number
listing_df["price"] = pd.to_numeric(listing_df["price"])
listing_df["weekly_price"] = pd.to_numeric(listing_df["weekly_price"])
listing_df["monthly_price"] = pd.to_numeric(listing_df["monthly_price"])
listing_df["security_deposit"] = pd.to_numeric(listing_df["security_deposit"])
listing_df["cleaning_fee"] = pd.to_numeric(listing_df["cleaning_fee"])
listing_df["extra_people"] = pd.to_numeric(listing_df["extra_people"])
listing_df.head()

,id,listing_name,street,neighbourhood_cleansed,zipcode,latitude,longitude,property_type,room_type,accommodates,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,host_id
0,12147973,Sunny Bungalow in the City,"Birch Street, Boston, MA 02131, United States",Roslindale,02131,42.282619,-71.133068,House,Entire home/apt,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,moderate,31303940
1,3075044,Charming room in pet friendly apt,"Pinehurst Street, Boston, MA 02131, United States",Roslindale,02131,42.286241,-71.134374,Apartment,Private room,2,...,10.0,9.0,10.0,10.0,9.0,9.0,False,True,moderate,2572247
2,6976,Mexican Folk Art Haven in Boston,"Ardale St., Boston, MA 02131, United States",Roslindale,02131,42.292438,-71.135765,Apartment,Private room,2,...,10.0,9.0,10.0,10.0,9.0,10.0,False,False,moderate,16701
3,1436513,Spacious Sunny Bedroom Suite in Historic Home,"Boston, MA, United States",Roslindale,NaN,42.281106,-71.121021,House,Private room,4,...,10.0,10.0,10.0,10.0,10.0,10.0,False,False,moderate,6031442
4,7651065,Come Home to Boston,"Durnell Avenue, Boston, MA 02131, United States",Roslindale,02131,42.284512,-71.136258,House,Private room,2,...,10.0,10.0,10.0,10.0,9.0,10.0,False,False,flexible,15396970


In [79]:
print(listing_df.columns)

Index(['id', 'listing_name', 'street', 'neighbourhood_cleansed', 'zipcode',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'price', 'weekly_price',
       'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included',
       'extra_people', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'cancellation_policy', 'host_id'],
      dtype='object')


## Create "Hosts" dataframe and clean it

In [60]:

# Create new data with select columns for the AirBnB hosts
host_df = listing_host_df[["host_id","host_name","host_since","host_location","host_response_time",
                           "host_response_rate","host_acceptance_rate","host_is_superhost",
                           "host_neighbourhood","host_listings_count","host_has_profile_pic",
                           "host_identity_verified"]].copy()

In [61]:
# delete duplicates
host_df.drop_duplicates(keep="first",inplace=True)

In [62]:
# convert dates to datetime format
host_df["host_since"]=pd.to_datetime(host_df["host_since"])

In [63]:
# convert boolean columns to boolean
host_df["host_is_superhost"].replace(["t","f"],[True,False],inplace=True)
host_df["host_has_profile_pic"].replace(["t","f"],[True,False],inplace=True)
host_df["host_identity_verified"].replace(["t","f"],[True,False],inplace=True)

In [64]:
# converting the percentage columns from strings to percentages
# remove the percentage symbols from the columns with percentages
host_df["host_response_rate"]=host_df["host_response_rate"].replace({'\$': '', ',': '', '%':''}, regex=True)
host_df["host_acceptance_rate"]=host_df["host_acceptance_rate"].replace({'\$': '', ',': '', '%':''}, regex=True)
# convert the percentage columns to numbers
host_df["host_response_rate"] = pd.to_numeric(host_df["host_response_rate"])
host_df["host_acceptance_rate"] = pd.to_numeric(host_df["host_acceptance_rate"])
host_df.head()

,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_has_profile_pic,host_identity_verified
0,31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",NaN,NaN,NaN,False,Roslindale,1,True,False
1,2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",within an hour,100.0,100.0,False,Roslindale,1,True,True
2,16701,Phil,2009-05-11,"Boston, Massachusetts, United States",within a few hours,100.0,88.0,True,Roslindale,1,True,True
3,6031442,Meghna,2013-04-21,"Boston, Massachusetts, United States",within a few hours,100.0,50.0,False,NaN,1,True,False
4,15396970,Linda,2014-05-11,"Boston, Massachusetts, United States",within an hour,100.0,100.0,True,Roslindale,1,True,True


In [65]:
print(host_df.columns)

Index(['host_id', 'host_name', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_has_profile_pic', 'host_identity_verified'],
      dtype='object')


## Create the "Reviews" dataframe and clean it

In [66]:
# Store CSV to DataFrame
reviews_file = "resources/reviews.csv"
reviews_df = pd.read_csv(reviews_file)
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


In [67]:
# change column "date/id" to something that SQL doesn't already recognize
reviews_df.rename(index=str,columns={"date":"review_date"},inplace=True)
reviews_df.rename(columns={"id":"review_id"}, inplace=True)

In [68]:
# Create new data with select columns for the AirBnB listings
review_df = reviews_df[['review_id', "listing_id", "review_date", "reviewer_id", "reviewer_name", "comments"]].copy()

In [69]:
# convert dates to datetime
review_df["review_date"] = pd.to_datetime(review_df["review_date"])
review_df.head()

,review_id,listing_id,review_date,reviewer_id,reviewer_name,comments
0,4724140,1178162,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,4869189,1178162,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,5003196,1178162,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,5150351,1178162,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,5171140,1178162,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


## Remove Neighborhoods that we're not using and clean relevant columns

In [80]:
listing_df.head()

,id,listing_name,street,neighbourhood_cleansed,zipcode,latitude,longitude,property_type,room_type,accommodates,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,host_id
0,12147973,Sunny Bungalow in the City,"Birch Street, Boston, MA 02131, United States",Roslindale,02131,42.282619,-71.133068,House,Entire home/apt,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,moderate,31303940
1,3075044,Charming room in pet friendly apt,"Pinehurst Street, Boston, MA 02131, United States",Roslindale,02131,42.286241,-71.134374,Apartment,Private room,2,...,10.0,9.0,10.0,10.0,9.0,9.0,False,True,moderate,2572247
2,6976,Mexican Folk Art Haven in Boston,"Ardale St., Boston, MA 02131, United States",Roslindale,02131,42.292438,-71.135765,Apartment,Private room,2,...,10.0,9.0,10.0,10.0,9.0,10.0,False,False,moderate,16701
3,1436513,Spacious Sunny Bedroom Suite in Historic Home,"Boston, MA, United States",Roslindale,NaN,42.281106,-71.121021,House,Private room,4,...,10.0,10.0,10.0,10.0,10.0,10.0,False,False,moderate,6031442
4,7651065,Come Home to Boston,"Durnell Avenue, Boston, MA 02131, United States",Roslindale,02131,42.284512,-71.136258,House,Private room,2,...,10.0,10.0,10.0,10.0,9.0,10.0,False,False,flexible,15396970


In [81]:
listing_df.groupby("neighbourhood_cleansed").count()

,id,listing_name,street,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,host_id
neighbourhood_cleansed,,,,,,,,,,,,,,,,,,,,,
Allston,260,260,260,258,260,260,259,260,260,260,...,181,182,182,182,181,182,260,260,260,260
Back Bay,302,302,302,299,302,302,301,302,302,302,...,236,236,236,236,236,235,302,302,302,302
Bay Village,24,24,24,24,24,24,24,24,24,24,...,16,16,16,16,16,16,24,24,24,24
Beacon Hill,194,194,194,194,194,194,194,194,194,194,...,168,168,168,168,168,168,194,194,194,194
Brighton,185,185,185,183,185,185,185,185,185,185,...,133,133,133,133,133,133,185,185,185,185
Charlestown,111,111,111,111,111,111,111,111,111,111,...,76,76,76,76,76,76,111,111,111,111
Chinatown,71,71,71,71,71,71,71,71,71,71,...,41,41,41,41,41,41,71,71,71,71
Dorchester,269,269,269,265,269,269,269,269,269,267,...,223,223,223,224,224,224,269,269,269,269
Downtown,172,172,172,170,172,172,171,172,172,172,...,118,118,118,118,118,118,172,172,172,172


In [83]:
# Restrict analysis to only neighborhoods of interest
listing_df = listing_df[listing_df["neighbourhood_cleansed"].isin(["Allston", "Fenway", 
                                                                         "Downtown", "Jamaica Plain"])]

In [84]:
# Remove rows that have 0 values for relevant columns
listing_df = listing_df[listing_df["price"] != 0]
listing_df = listing_df[listing_df["cancellation_policy"] != 0]
listing_df = listing_df[listing_df["room_type"] != 0]
listing_df = listing_df[listing_df["review_scores_value"] != 0]
listing_df = listing_df[listing_df["number_of_reviews"] != 0]

# Remove rows that have N/A for relevant columns
listing_df.dropna(subset=["price","cancellation_policy","room_type",
                          "review_scores_value","number_of_reviews","bedrooms","bathrooms"],how="any",inplace=True)

In [86]:
# Reduce the host dataframe to only the relevant neighborhoods
host_reduced_df = pd.merge(listing_df,host_df,how="left",left_on=["host_id"],right_on=["host_id"])
host_reduced_df.dropna(subset=["price","cancellation_policy","room_type",
                               "review_scores_value","number_of_reviews"],how="any",inplace=True)

host_reduced_df = host_reduced_df[["host_id","host_name","host_since","host_location","host_response_time","host_response_rate",
                                   "host_acceptance_rate","host_is_superhost","host_neighbourhood","host_listings_count","host_has_profile_pic",
                                   "host_identity_verified"]]

host_reduced_df.drop_duplicates(keep="first",inplace=True)

In [87]:
# Reduce the reviews dataframe to only the relevant neighborhoods
reviews_reduced_df = pd.merge(reviews_df,listing_df,how="left",left_on=["listing_id"],right_on=["id"])

reviews_reduced_df.dropna(subset=["price","cancellation_policy","room_type","review_scores_value",
                                  "number_of_reviews"],how="any",inplace=True)

reviews_reduced_df = reviews_reduced_df[['review_id', "listing_id", "review_date", "reviewer_id", 
                                         "reviewer_name", "comments"]]


reviews_reduced_df.drop_duplicates(keep="first",inplace=True)

## SQL

In [126]:
#create database connection
connection = "postgres:postgres@localhost:5432/airbnb_db"
engine = create_engine(f'postgresql://{connection}')

In [127]:
# check tablenames in the MySQL database
engine.table_names()

['airbnb_hosts', 'listings', 'property_reviews', 'reviews']

In [131]:
# add the hosts dataframe to MySQL
host_reduced_df.to_sql(name="airbnb_hosts",con=engine,if_exists="append",index=False,chunksize=2000)

In [132]:
# add the listings dataframe to MySQL
listing_df.to_sql(name="listings",con=engine,if_exists="append",index=False,chunksize=200)

In [134]:
# add the property reviews dataframe to MySQL
reviews_reduced_df.to_sql(name="property_reviews",con=engine,if_exists="append",index=False,chunksize=2000)

In [139]:
host_reduced_df.to_csv("./resources/reduced_host.csv",index=False)

In [140]:
listing_df.to_csv("./resources/cleaned_listing.csv",index=False)

In [141]:
reviews_reduced_df.to_csv("./resources/cleaned_review.csv",index=False)

###Sanity Check

In [135]:
pd.read_sql_query('select * from listings', con=engine).head()

,id,listing_name,street,neighbourhood_group_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,cancellation_policy,host_id,neighbourhood_cleansed
0,594693,Private room in peaceful space,"South St, Boston, MA 02130, United States",None,02130,42.307975,-71.116221,None,Apartment,Private room,...,8,10,10.0,10.0,9,False,False,strict,2139546,Jamaica Plain
1,2384581,"Loft bedroom, sleeps 2","Holbrook Street, Boston, MA 02130, United States",None,02130,42.309357,-71.116827,None,House,Private room,...,9,10,10.0,10.0,9,False,False,flexible,12083797,Jamaica Plain
2,735970,Part of a Large Condo Jamaica Plain,"Beaufort Rd, Boston, MA 02130, United States",None,02130,42.316912,-71.113025,None,Bed & Breakfast,Private room,...,9,9,9.0,10.0,10,False,False,flexible,3840038,Jamaica Plain
3,4262255,Beautiful and sunny condo,"Rossmore Road, Boston, MA 02130, United States",None,02130,42.303985,-71.104494,None,Apartment,Private room,...,10,10,10.0,10.0,10,False,False,moderate,21873698,Jamaica Plain
4,4000384,Sunlit room with private bath,"Copley Street, Boston, MA 02119, United States",None,02119,42.316329,-71.099096,None,House,Private room,...,10,10,10.0,9.0,10,False,False,flexible,10910804,Jamaica Plain


In [137]:
pd.read_sql_query('select * from airbnb_hosts', con=engine).head()

,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_has_profile_pic,host_identity_verified
0,2139546,Robin,2012-04-14,"Boston, Massachusetts, United States",within an hour,100.0,100.0,False,Jamaica Plain,2,True,False
1,12083797,Molly,2014-02-09,"Boston, Massachusetts, United States",within a day,93.0,73.0,False,Jamaica Plain,2,True,True
2,3840038,Sidney,2012-10-11,"Boston, Massachusetts, United States",None,NaN,NaN,False,Jamaica Plain,1,True,False
3,21873698,Elizabeth,2014-09-28,"Boston, Massachusetts, United States",within a few hours,100.0,89.0,True,Jamaica Plain,1,True,True
4,10910804,Evelyn,2013-12-30,"Boston, Massachusetts, United States",within a day,70.0,85.0,False,Jamaica Plain,2,True,True


In [138]:
pd.read_sql_query('select * from property_reviews', con=engine).head()

,review_id,listing_id,review_date,reviewer_id,reviewer_name,comments
0,57999038,9364092,2015-12-31,1211289,Hani,The host canceled this reservation 13 days bef...
1,58979667,9364092,2016-01-05,51948821,Julia,Sergio has a lovely and artistic home. The amb...
2,62651888,9364092,2016-02-15,47402064,Kathryn,"Sergio was a fantastic host. He is warm, frie..."
3,70617954,9364092,2016-04-19,37224180,André,Sergio is a fantastic host ! \r\nWe had a grea...
4,92507365,9364092,2016-08-08,26023372,Cheri,Sergio was a fantastic host. Our flight was de...
